In [ ]:
#用于算法交易的神经网络——正确的时间序列预测和回测


In [1]:
import matplotlib.pylab as plt

In [2]:
import seaborn as sns

In [3]:
sns.despine()

In [4]:
from keras.models import Sequential

Using TensorFlow backend.


In [5]:
from keras.layers.core import Dense, Dropout, Activation, Flatten


In [6]:
from keras.layers.normalization import BatchNormalization


In [7]:
from keras.layers import merge


In [8]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger, EarlyStopping


In [9]:
from keras.optimizers import RMSprop, Adam, SGD, Nadam


In [10]:
from keras.layers.advanced_activations import *


In [11]:
from keras.layers import Convolution1D, MaxPooling1D, AtrousConvolution1D


In [12]:
from keras.layers.recurrent import LSTM, GRU


In [13]:
from keras import regularizers

In [14]:
import theano


In [15]:
theano.config.compute_test_value = "ignore"

In [16]:
from sklearn.cross_validation import train_test_split

ModuleNotFoundError: No module named 'sklearn.cross_validation'

In [18]:
import pandas as pd
import numpy as np

In [19]:
def shuffle_in_unison(a, b):
    # courtsey http://stackoverflow.com/users/190280/josh-bleecher-snyder
    assert len(a) == len(b)
    shuffled_a = np.empty(a.shape, dtype=a.dtype)
    shuffled_b = np.empty(b.shape, dtype=b.dtype)
    permutation = np.random.permutation(len(a))
    for old_index, new_index in enumerate(permutation):
        shuffled_a[new_index] = a[old_index]
        shuffled_b[new_index] = b[old_index]
    return shuffled_a, shuffled_b

In [20]:
# 将数据的前90%作为训练集，后10%作为测试集
def create_Xt_Yt(X, y, percentage=0.9):
    p = int(len(X) * percentage)
    X_train = X[0:p]
    Y_train = y[0:p]
     
    X_train, Y_train = shuffle_in_unison(X_train, Y_train)
 
    X_test = X[p:]
    Y_test = y[p:]

    return X_train, X_test, Y_train, Y_test



In [21]:
#训练起始时间
start_date='2010-01-01'
end_date='2016-02-01'
#inst = D.instruments(start_date, end_date, market='CN_STOCK_A')
#print(inst)
instruments = ['600519.SHA']
hist = D.history_data(instruments, start_date, end_date, fields=['close'])

print(hist.head())

plt.plot(hist['date'], hist['close'])
plt.show()

NameError: name 'D' is not defined

In [22]:
# 转化为时间序列数据
hist = hist.ix[:, 'close'].tolist()

WINDOW = 30
EMB_SIZE = 1
STEP = 1
FORECAST = 1

NameError: name 'hist' is not defined

In [23]:
# Straightforward way for creating time windows
X, Y = [], []
for i in range(0, len(hist), STEP): 
    try:
        x_i = hist[i:i+WINDOW]
        y_i = hist[i+WINDOW+FORECAST]  

        last_close = x_i[WINDOW-1]
        next_close = y_i

        if last_close < next_close:
            y_i = [1, 0]
        else:
            y_i = [0, 1] 

    except Exception as e:
        print(e)
        break

    X.append(x_i)
    Y.append(y_i)

X = [(np.array(x) - np.mean(x)) / np.std(x) for x in X] # comment it to remove normalization
X, Y = np.array(X), np.array(Y)

#X_train, X_test, Y_train, Y_test = create_Xt_Yt(X, Y)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

model = Sequential()
model.add(Dense(64, input_dim=WINDOW,
                activity_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(LeakyReLU())

NameError: name 'hist' is not defined

In [24]:
# 加入 Dropout 防止过拟合
model.add(Dropout(0.5))
model.add(Dense(16, activity_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(Dense(2))
model.add(Activation('softmax'))

opt = Nadam(lr=0.001)

reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.9, patience=25, min_lr=0.000001, verbose=1)
checkpointer = ModelCheckpoint(filepath="test.hdf5", verbose=1, save_best_only=True)
early_stopper = EarlyStopping(monitor='val_loss', min_delta=0.05, patience=20, verbose=0, mode='auto')
model.compile(optimizer=opt, 
              loss='categorical_crossentropy',
              metrics=['accuracy'])



NameError: name 'model' is not defined

In [25]:
# Train model
history = model.fit(X_train, Y_train, nb_epoch = 100, batch_size = 128, verbose=1, validation_data=(X_test, Y_test), shuffle=True, callbacks=[reduce_lr, checkpointer, early_stopper])



NameError: name 'model' is not defined

In [26]:
# Visualization of loss
plt.figure() 
plt.plot(history.history['loss']) 
plt.plot(history.history['val_loss']) 
plt.title('model loss') 
plt.ylabel('loss') 
plt.xlabel('epoch') 
plt.legend(['train', 'test'], loc='best') 

NameError: name 'history' is not defined

In [27]:
# Visualization of Accuracy
plt.figure()
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='best')
plt.show()



NameError: name 'history' is not defined

In [28]:
# 1. 策略基本参数

# 回测起始时间
start_date_2 = '2016-02-02'
# 回测结束时间
end_date_2 = '2017-06-20'
# 策略比较参考标准，以沪深300为例
benchmark = '000300.INDX'
# 证券池 以贵州茅台为例
# instruments = ['000030.SZA']
# 起始资金
capital_base = 100000

# 2. 策略主体函数

# 初始化虚拟账户状态，只在第一个交易日运行
def initialize(context):
    # 设置手续费，买入时万3，卖出是千分之1.3,不足5元以5元计
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))

# 策略交易逻辑，每个交易日运行一次
def handle_data(context, data):
    # 在这里添加策略代码
    for instrument in instruments:
        # 字符型股票代码转化成 BigQuant回测引擎所需的股票代码
        instrument = context.symbol(instrument)
        history = data.history(instrument, 'close', 30, '1d').tolist()
        norm_hist = [(np.array(history) - np.mean(history)) / np.std(history)]
        prediction = model.predict(np.asarray(norm_hist))[0]
        
        #print(prediction)
        
        if  np.argmax(prediction) == 0 and data.can_trade(instrument):
            order_target_percent(instrument, 1)
        elif np.argmax(prediction) == 1 and data.can_trade(instrument):
            order_target_percent(instrument, 0)

# 3. 启动回测

# 策略回测接口: https://bigquant.com/docs/module_trade.html
m = M.trade.v1(
    instruments=instruments,
    start_date=start_date_2,
    end_date=end_date_2,
    initialize=initialize,
    handle_data=handle_data,
    # 买入订单以开盘价成交
    order_price_field_buy='open',
    # 卖出订单以开盘价成交
    order_price_field_sell='open',
    capital_base=capital_base,
    benchmark=benchmark,
)
# 调用风险分析
m.risk_analyze()  



NameError: name 'M' is not defined